In [88]:
## PATH settings
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(project_root)
COMMON_PATH = os.path.join(project_root, 'common')

In [89]:
## Bespoke class imports
from common.database.adatabase import ADatabase
from common.processor.processor import Processor as p
from common.extractor.alpaca_extractor import AlpacaExtractor
from financial_common.indicator.indicator import Indicator
from financial_common.risk.benchmark import Benchmark
from financial_common.portfolio_management.kpi import KPI
from financial_common.metric.metric import Metric
from financial_common.portfolio_management.portfolio import Portfolio
import numpy as np
import matplotlib.pyplot as plt

In [90]:
## Import standard packages
from datetime import timedelta, datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from time import sleep
import copy

In [91]:
orivault = ADatabase("ori")
alp = AlpacaExtractor(paper=False)

In [92]:
orivault.connect()
portfolio_dictionary = orivault.retrieve("results").to_dict("records")[0]
orivault.disconnect()
portfolio_dictionary

{'ranking_metric': 'ema',
 'timeframe': 'DAY',
 'position_type': 'long',
 'grouping_type': 'STANDARD_DEV',
 'selection_type': 'top_blacklist',
 'allocation_type': 'risk',
 'risk_type': 'coefficient_of_variation',
 'selection_percentage': 1,
 'stoploss': 1,
 'num_of_groups': 1,
 'max_price': 0.1,
 'min_price': 0.01,
 'max_market_cap': 1000,
 'min_market_cap': 0,
 'rolling_window': 10,
 'leverage': 1,
 'date': Timestamp('2025-03-14 00:00:00'),
 'pnl': 4.1950354744169465,
 'raw_pnl': 5.089981622039429,
 'downside': 1.2983208945147313,
 'coefficient_of_variation': 3.6605559025332273,
 'portfolio_std': 1.1460104929728956,
 'sharpe_ratio': 2.7871313499346337,
 'tracking_error': 1.1463677187002028,
 'information_ratio': 2.786262837146512}

In [93]:
pm = Portfolio.from_dict(portfolio_dictionary)
pm.to_dict()

{'ranking_metric': 'ema',
 'timeframe': 'DAY',
 'position_type': 'long',
 'grouping_type': 'STANDARD_DEV',
 'selection_type': 'top_blacklist',
 'allocation_type': 'risk',
 'risk_type': 'coefficient_of_variation',
 'selection_percentage': 1.0,
 'stoploss': 1.0,
 'num_of_groups': 1,
 'max_price': 0.1,
 'min_price': 0.01,
 'max_market_cap': 1000.0,
 'min_market_cap': 0.0,
 'rolling_window': 10,
 'leverage': 1}

In [94]:
## Retrieve existing stocks 
index = alp.assets().sort_values("ticker")
tickers = index["ticker"].values
tickers_per_batch = int(1000)
batchs = [tickers[i:i + tickers_per_batch] for i in range(0, len(tickers), tickers_per_batch)]
bars = []
[bars.append(alp.latest_bars_bulk(batch)) for batch in batchs]
todays_bar = pd.concat(bars)
index = index.merge(todays_bar,on="ticker",how="left")

'SVA'


In [95]:
end = alp.clock() - timedelta(days=1)
start = (end - timedelta(days=pm.rolling_window*3))
print(start,end)

2025-04-25 13:34:05.466443-04:00 2025-05-25 13:34:05.466443-04:00


In [96]:
tickers_per_batch = int(10000/(pm.rolling_window*4)/5)
print(tickers_per_batch)

50


In [97]:
relevant_tickers = index[(index["adjclose"]>=pm.min_price) & (index["adjclose"]<=pm.max_price)]["ticker"]
batchs = [relevant_tickers[i:i + tickers_per_batch] for i in range(0, len(relevant_tickers), tickers_per_batch)]

In [98]:
prices = []
for batch in tqdm(batchs):
    try:
        tickers_data = alp.prices_bulk(batch,start,end)
        for ticker in batch:
            try:
                price = tickers_data[tickers_data["ticker"] == ticker].copy()
                bar = todays_bar[todays_bar["ticker"]==ticker][["date","ticker","adjclose","high","low","volume"]].copy()
                price = p.lower_column(price)
                price = pd.concat([price,bar])
                price = p.utc_date(price)
                price.sort_values("date", inplace=True)
                price = p.additional_date_columns(price)
                price["market_cap"] = price["adjclose"] * price["volume"]
                if price[(price["adjclose"]<=pm.max_price) & (price["adjclose"]>=pm.min_price)].index.size > 0:
                    price["standard_dev"] = 1
                    price["coefficient_of_variation"] = 1
                    price = Indicator.indicator_type_factory(pm.ranking_metric).calculate(price,timeframe=pm.rolling_window,live=True)
                    prices.append(price)
            except Exception as e:
                print(str(e))
    except Exception as e:
        print(str(e))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.39it/s]


In [100]:
simulation = pd.concat(prices)
simulation.sort_values("date", inplace=True)
trades = pm.recs(simulation.copy())
trades

,year,day,ticker,date,adjclose,high,low,volume,month,quarter,...,weekday,market_cap,standard_dev,risk,ema,group_percentile,rank_percentile,major_key,position_type,weight
20,2025,143,HYMCL,2025-05-23 00:00:00+00:00,0.0106,0.0228,0.0103,85062,5,2,...,4,901.6572,1,1,1.47851,1,1,2025_143_1,1.0,1.0


In [101]:
options = alp.call_options("GDHG",datetime(2025,5,23))
options

dict_keys([])

In [102]:
orivault.connect()
orivault.drop("recommendations")
orivault.store("recommendations",trades)
orivault.disconnect()